# 序列的修改、散列和切片

In [1]:
from array import array
import reprlib
import math

In [ ]:
class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)])) + bytes(self._components)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [ ]:
# 协议和鸭子类型
# python序列协议只需要__len__和__getitem__两个方法
# 为了支持迭代，只需要实现__getitem__方法，没必要提供len方法

In [9]:
# Vector 类第二版: 可切片的序列
class Vector:
    typecode = 'd'
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    # ...
    
    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        return self._components[index]

In [10]:
v1 = Vector([3,4,5])

In [12]:
# 切片是实现了，但是返回的是数组，需要返回vector
print (v1[1:4])

array('d', [4.0, 5.0])

In [14]:
# 切片原理
class MySeq:
    def __getitem__(self, index):
        return index

In [15]:
s = MySeq()
print (s[1])
print (s[1:4])
print (s[1:4:2])
print (s[1:4:2, 9])
print (s[1:4:2, 7:9])

1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


In [16]:
print (slice)

<class 'slice'>


In [17]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [18]:
print (slice(None, 10, 2).indices(5))
print (slice(-3, None, None).indices(5))

(0, 5, 2)
(2, 5, 1)


In [25]:
import numbers
import functools
import operator

In [ ]:
# 能处理切片的__getitem__方法
# 注意，vector不支持多维索引，因此索引元祖或多个切片会抛出错误
class Vector:
    typecode = 'd'
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    # ...
    
    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

In [ ]:
# 动态存取属性
class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    # ...
    
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name = cls.__name__, attr_name = name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

    # hash
    def __eq__(self, other):
        return tuple(self) == tuple(other)

    #为了提高效率
    def __eq__(self, other):
        if len(self) != len(other):
            return False
        for a, b in zip(self, other):
            if a != b:
                return False
        return True
    
    # 更简洁的写法
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))

    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)